In [1]:
import os
# Force bitsandbytes to use the CUDA modules it finds in its own folder
os.environ["BNB_CUDA_VERSION"] = "121" # Or 118, depending on Kaggle's current CUDA
os.environ["LD_LIBRARY_PATH"] += ":/usr/local/cuda/lib64"

In [2]:
# ==========================================
# 1. PROVEN OFFLINE INSTALL (Original Method)
# ==========================================
import os
import torch
import gc
import site
import json
import pandas as pd
from tqdm import tqdm

# Cleanup memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

LIB_PATH = "offline_libs"
ZIP_PATH = "/kaggle/input/zno-libs-final/offline_libs.zip"

if not os.path.exists(LIB_PATH):
    if os.path.exists(ZIP_PATH):
        print("📦 Unzipping original libraries...")
        !unzip -q {ZIP_PATH} -d .
    elif os.path.exists("/kaggle/input/zno-libs-final/offline_libs"):
        LIB_PATH = "/kaggle/input/zno-libs-final/offline_libs"

print(f"📦 Installing from {LIB_PATH}...")
# Reverting to your exact install command
!pip install --no-index --find-links={LIB_PATH} bitsandbytes peft accelerate transformers > /dev/null

site.main()
print("✅ Installation Complete!")

# ==========================================
# 2. CONFIGURATION & IMPORTS
# ==========================================
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

BATCH_SIZE = 1           
MAX_CONTEXT_LEN = 1500   # Optimized to fit full examples
BASE_MODEL_PATH = "/kaggle/input/qwen2.5/transformers/7b-instruct/1" 
ADAPTER_PATH = "/kaggle/input/zno-my-adapter"

# Find test data path
TEST_FILE_PATH = "/kaggle/input/zno-data/zno.test.jsonl"
if not os.path.exists(TEST_FILE_PATH):
    for root, _, files in os.walk("/kaggle/input"):
        if "zno.test.jsonl" in files:
            TEST_FILE_PATH = os.path.join(root, "zno.test.jsonl")

# ==========================================
# 3. LOAD MODEL (Stable 4-bit)
# ==========================================
print(f"⏳ Loading Model (4-bit)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",
    local_files_only=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, local_files_only=True)
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token 

print(f"🔗 Attaching Adapter...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH, local_files_only=True)
model.eval()

# ==========================================
# 4. BALANCED LOGIT SCORING (Score Optimizer)
# ==========================================
# Mapping Cyrillic letters to model vocabulary
candidates = ["А", "Б", "В", "Г", "Д"]
candidate_ids = [tokenizer.encode(c, add_special_tokens=False)[-1] for c in candidates]
answer_map = {0: "А", 1: "Б", 2: "В", 3: "Г", 4: "Д"}

# Balanced few-shot (Answers are A, B, V, G to eliminate letter bias)
BALANCED_EXAMPLES = """
Питання: Слово з орфографічною помилкою є в рядку
Варіанти:
А: контрастний, баластний, форпостний
Б: пестуни, хвастливий, кістлявий
В: студентський, диригентський, туристський
Г: тижневий, серцевий, злісний
Д: учасник, яхтсмен, стільниковий
Відповідь: А

Питання: Правильно написано всі слова в рядку
Варіанти:
А: сфотографувати, розжувати, припрілий
Б: беззахисний, схибити, розчин
В: перекотиполе, незрівняний, припинити
Г: превелебний, сказати, безсмертя
Д: спитати, розчесати, прирва
Відповідь: Б

Питання: Укажіть рядок, у якому всі слова пишуться з великої літери
Варіанти:
А: (Ш,ш)евченківські вірші, (К,к)иївські вулиці
Б: (Д,д)ніпровські хвилі, (Л,л)ьвівська кава
В: (П,п)івденний (Б,б)уг, (З,з)олоті (В,в)орота
Г: (Н,н)аціональний (Б,б)анк, (В,в)ерховна (Р,р)ада
Д: (М,м)іністерство (О,о)світи, (К,к)абмін
Відповідь: В

Питання: Укажіть правильний варіант послідовного заповнення пропусків
Варіанти:
А: найбільш цікавішим, по самі вуха, в анфас
Б: якнайцікавішим, по вуха, в анфас
В: цікавішим, по самі вуха, анфас
Г: найбільш цікавим, по самі вуха, в анфас
Д: найцікавішим, по вуха, анфас
Відповідь: Г
"""

def create_prompt(item):
    q = item.get('question', '')
    if 'answers' in item:
        opts = "\n".join([f"{opt['marker']}: {opt['text']}" for opt in item['answers']])
    else:
        opts = str(item.get('answers', ''))
    
    instruction = "Дай відповідь буквою-варіантом відповіді з наданих варіантів."
    return f"<|im_start|>user\n{instruction}\n\n{BALANCED_EXAMPLES}\n\nПитання: {q}\nВаріанти:\n{opts}<|im_end|>\n<|im_start|>assistant\nВідповідь:"

# ==========================================
# 5. EXECUTION
# ==========================================
test_data = []
with open(TEST_FILE_PATH, "r", encoding="utf-8") as f:
    for line in f:
        try: test_data.append(json.loads(line))
        except: pass

print(f"🚀 Starting Optimized Run on {len(test_data)} items...")
results = []

for i in tqdm(range(0, len(test_data), BATCH_SIZE)):
    if i % 50 == 0: torch.cuda.empty_cache()
    
    batch_items = test_data[i : i + BATCH_SIZE]
    batch_prompts = [create_prompt(item) for item in batch_items]
    batch_ids = [item.get('id') for item in batch_items]

    inputs = tokenizer(
        batch_prompts, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=MAX_CONTEXT_LEN
    ).to("cuda")

    with torch.inference_mode(): 
        outputs = model(**inputs)
        # Higher score: We compare the probability of each answer letter at the decision token
        next_token_logits = outputs.logits[:, -1, :]
        candidate_scores = next_token_logits[:, candidate_ids]
        best_indices = torch.argmax(candidate_scores, dim=1).cpu().numpy()

    for q_id, idx in zip(batch_ids, best_indices):
        results.append({"id": q_id, "answer": answer_map[idx]})

pd.DataFrame(results).to_csv("submission.csv", index=False)
print(f"✅ Submission created with higher precision strategy!")

📦 Installing from /kaggle/input/zno-libs-final/offline_libs...
✅ Installation Complete!


2026-01-13 21:18:09.536736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768339089.722283      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768339089.772364      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768339090.198023      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768339090.198061      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768339090.198064      55 computation_placer.cc:177] computation placer alr

⏳ Loading Model (4-bit)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

🔗 Attaching Adapter...


/usr/local/lib/python3.12/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['alora_invocation_tokens', 'arrow_config', 'ensure_weight_tying', 'peft_version', 'use_bdlora'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


🚀 Starting Optimized Run on 751 items...


  2%|▏         | 12/751 [00:29<29:51,  2.42s/it]


KeyboardInterrupt: 